# Purpose
This notebook describes the typical activities carried out  at the beginning to a project / thread when customer shares new data. We will be trying to understand the tables, columns and information flow. Typically we also look for data issues and confirm with respective owners for resolution. At the end of this activity, the data sources and their treatment is finalized. Code in this notebook will not be part of the production code.

This data can be downloaded from
[here](https://drive.google.com/file/d/11DqcBxxEcn3QA4YvPQmmExBm-m6AgUQ_/view?usp=sharing)

**NOTE**:
Download the data from the above link, and copy the extracted csv files to the path `data/raw/sales/` (relative to root of the code archive folder). Make sure to copy the files before continuing on with the rest of the notebook.

# Initialization

In [56]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
%%time

# Third-party imports
import os.path as op
import pandas as pd
import great_expectations as ge

# Project imports
from ta_lib.core.api import display_as_tabs, initialize_environment

# Initialization
initialize_environment(debug=False, hide_warnings=True)

CPU times: user 132 µs, sys: 14 µs, total: 146 µs
Wall time: 150 µs


# Data

## Background

Customer is a distributor of electronic devices. They partner with manufacturers, carriers and refurbishers and sell across to  retailers. The selling price is the outcome of negotiation between sales representatives and retailers. Customer wants to understand the selling price variation and determine  optimal pricing with Machine Learning.

In [58]:
from ta_lib.core.api import create_context, list_datasets, load_dataset

In [59]:
config_path = op.join('conf', 'config.yml')
context = create_context(config_path)

In [60]:
list_datasets(context)

['/raw/housing',
 '/cleaned/housing',
 '/processed/housing',
 '/train/housing/features',
 '/train/housing/target',
 '/test/housing/features',
 '/test/housing/target',
 '/score/housing/output']

In [61]:
# load datasets
housing_df = load_dataset(context, '/raw/housing')

# Exploratory Analysis

Given the raw data from data ingestion, we would now like to explore and learn more details about the data.


The output of the step would be a summary report and discussion of any pertinent findings.


In [62]:
# Import the eda API
import ta_lib.eda.api as eda

## Variable summary

In [63]:
display_as_tabs([('housing', housing_df.shape)])

Tabs
    [0] Str(tuple, name='housing')

In [64]:
sum1 = eda.get_variable_summary(housing_df)

display_as_tabs([('housing', sum)])

Tabs
    [0] DataFrame(name='housing', value=              ...)

In [65]:
housing_df.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

**Dev NOTES**

<details>
1. Datatypes : We have both numeric and other types. The bulk of them seem to be numeric. `Numeric` is defined to be one of [float|int|date] and the rest are categorized as `Others`. A column is assumed to have `date` values if it has the string `date` in the column name.


### Summary Plot

Provides a high level summary of the dataset health.

**Watch out for:**

* too few numeric values
* high % of missing values
* high % of duplicate values
* high % of duplicate columns 

In [66]:
sum, plot1 = eda.get_data_health_summary(housing_df, return_plot=True)

display_as_tabs([('housing', plot1)])

Tabs
    [0] HoloViews(Layout, name='housing')

**Dev NOTES**

<details>
1. Datatypes : We have both numeric and other types. The bulk of them seem to be numeric. `Numeric` is defined to be one of [float|int|date] and the rest are categorized as `Others`. A column is assumed to have `date` values if it has the string `date` in the column name.

2. The missing value plot seems to indicate missing values are not present but we do have them. 

3. We are looking for duplicate observations (rows in the data). The plot shows the % of rows that are an exact replica of another row (using `df.duplicated`)

4. We are looking for duplicate features (columns in the data).

</details>

### Missing Values summary

This provides an overall view focussing on amount of missing values in the dataset.

**Watch out for:**
* A few columns have significant number of missing values 
* Most columns have significant number of missing values


In [67]:
sum1, plot1 = eda.get_missing_values_summary(housing_df, return_plot=True)

display_as_tabs([('housing', plot1)])

Tabs
    [0] HoloViews(Bars, name='housing')

**Dev notes:**

<details>
    
    * By default, the following are considered missing/NA values : `[np.Nan, pd.NaT, 'NA', None]`
    * additional values can be passed to tigerml (add_additional_na_values)
    * these are applied to all columns.
    
    * some of the above information can be learnt from the data discovery step (see discussion below)
    
</details>

In [68]:
sum1 = eda.get_duplicate_columns(housing_df)

display_as_tabs([('housing', sum1)])

Tabs
    [0] Markdown(str, name='housing')

In [69]:
sum1 = eda.get_outliers(housing_df)

display_as_tabs([('housing', sum1)])

Tabs
    [0] DataFrame(name='housing', value=                    ('Data...)

## Health Analysis report

Generate a report that has all the above data in a single html. This could be useful to submit to a client

In [70]:
from ta_lib.reports.api import summary_report

summary_report(housing_df, './housing.html')